In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
tokenizer = Tokenizer()

data = ''' Yeh chand sa roshan chehra
julfo ka rang sunehra
Yeh jheel si neeli aankhein
Koi raaz hai inmein gehra
Tareef karu kya uski
jisne tumhein banaya
Ek cheez qayamat si hai
Logon se suna karte the
Tumhein dekh ke maine mana
Woh theek kaha karte the
Hai chaal mein teri zalim
Kuchh aisi balaa ka jaadu
Sau baar sambhala dil ko
Par hoke raha bekabu
Har subah kiran ki laali
hai rang tere gaalon ka
Har shaam ki chadar kaali
saya hai tere baalon ka
Tu balkhaati ek nadiya
har mauj teri angdaayi
Jo in maujon mein dooba
usne hi duniya paayi
Main khoj mein hoon manzil ke
Aur manzil paas hai mere
Mukhde se hata do aanchal
Ho jaayein door andhere
Mana ke yeh jalwe tere
Kar denge mujhe deewana
Jee bhar ke zara main dekhoon
Andaz tera mastana
'''
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)+1

In [4]:
import numpy as np

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_seq = token_list[:i+1]
        input_sequences.append(n_gram_seq)
# this is so that we can predict the next word , i.e if we see these
# 3 words , what should be the 4th word ??

# Now padding
input_sequences = np.array(pad_sequences(input_sequences))
print(input_sequences)

[[  0   0   0   0   4  20]
 [  0   0   0   4  20  21]
 [  0   0   4  20  21  22]
 [  0   4  20  21  22  23]
 [  0   0   0   0  24   2]
 [  0   0   0  24   2   8]
 [  0   0  24   2   8  25]
 [  0   0   0   0   4  26]
 [  0   0   0   4  26   9]
 [  0   0   4  26   9  27]
 [  0   4  26   9  27  28]
 [  0   0   0   0  29  30]
 [  0   0   0  29  30   1]
 [  0   0  29  30   1  31]
 [  0  29  30   1  31  32]
 [  0   0   0   0  33  34]
 [  0   0   0  33  34  35]
 [  0   0  33  34  35  36]
 [  0   0   0   0  37  10]
 [  0   0   0  37  10  38]
 [  0   0   0   0  11  39]
 [  0   0   0  11  39  40]
 [  0   0  11  39  40   9]
 [  0  11  39  40   9   1]
 [  0   0   0   0  41  12]
 [  0   0   0  41  12  42]
 [  0   0  41  12  42  13]
 [  0  41  12  42  13  14]
 [  0   0   0   0  10  43]
 [  0   0   0  10  43   3]
 [  0   0  10  43   3  44]
 [  0  10  43   3  44  15]
 [  0   0   0   0  45  46]
 [  0   0   0  45  46  47]
 [  0   0  45  46  47  13]
 [  0  45  46  47  13  14]
 [  0   0   0   0   1  48]
 

In [5]:
xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels,num_classes=total_words)

In [6]:
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
max_sequence_len = max(len(x) for x in input_sequences)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words,240,input_length=max_sequence_len-1),
])
model.add(Bidirectional(LSTM(150)))
model.add(Dense(3*total_words,activation='relu'))
model.add(Dense(2*total_words,activation='relu'))
model.add(Dense(total_words,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(
    xs,ys,
    epochs=100,
)

2023-01-03 11:39:30.491006: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-01-03 11:39:31.414370: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
4/4 [==============================] - 4s 22ms/step - loss: 4.7013 - accuracy: 0.0091
Epoch 2/100
4/4 [==============================] - 0s 18ms/step - loss: 4.6827 - accuracy: 0.0545
Epoch 3/100
4/4 [==============================] - 0s 19ms/step - loss: 4.6555 - accuracy: 0.0818
Epoch 4/100
4/4 [==============================] - 0s 18ms/step - loss: 4.6053 - accuracy: 0.0636
Epoch 5/100
4/4 [==============================] - 0s 18ms/step - loss: 4.5144 - accuracy: 0.0727
Epoch 6/100
4/4 [==============================] - 0s 19ms/step - loss: 4.4584 - accuracy: 0.0636
Epoch 7/100
4/4 [==============================] - 0s 18ms/step - loss: 4.3897 - accuracy: 0.0636
Epoch 8/100
4/4 [==============================] - 0s 19ms/step - loss: 4.3413 - accuracy: 0.0636
Epoch 9/100
4/4 [==============================] - 0s 19ms/step - loss: 4.2849 - accuracy: 0.0727
Epoch 10/100
4/4 [==============================] - 0s 19ms/step - loss: 4.2017 - accuracy: 0.0636
Epoch 11/100
4/4 [=

In [7]:
# GENERATING TEXT
seed_text = "Yeh chand sa roshan chehra"
next_words = 100

for _ in range(next_words):
    seed_seq = tokenizer.texts_to_sequences([seed_text.lower()])[0]
    seed_seq = pad_sequences([seed_seq],maxlen=max_sequence_len-1,padding='pre')
    predicted = np.argmax(model.predict(seed_seq),axis=1)
    output_word= ""
    for word,index in tokenizer.word_index.items():
        if(index==predicted):
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

Yeh chand sa roshan chehra chehra chehra kaali kaali chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra chehra
